In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [19]:
torch.manual_seed(1)

## Softmax

In [20]:
z = torch.FloatTensor([1,2,3])

hypothesis = F.softmax(z, dim=0)
print(hypothesis) # 각각의 확률값

tensor([0.0900, 0.2447, 0.6652])


In [21]:
# propability
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss(Low-level)

In [22]:
z = torch.rand(3,5, requires_grad = True)
hypothesis = F.softmax(z, dim=1) # 두 번째 dimension에 대해서 구함
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [23]:
# 임의의 정답 인덱스
y = torch.randint(5,(3,)).long()
print(y)

tensor([0, 2, 1])


In [24]:
print(y.shape) # (1,3)
print(y.unsqueeze(1)) # (3,1)

torch.Size([3])
tensor([[0],
        [2],
        [1]])


In [25]:
# 정답 원핫인코딩
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1),1) # inplace 연산,

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [26]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean() # (3,5) -> (3,) -> scalar  
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## Cross Entropy Loss with torch.nn.functional

In [32]:
F.softmax(z,dim=1)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)

In [27]:
# low level
torch.log(F.softmax(z,dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [28]:
# high level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

### F.nll_loss() - negative loss likelihood

In [30]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [29]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [31]:
# log_softmax + nll_loss
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Training with Low-lebel Cross Entorpy Loss

In [33]:
# (m,4) data
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0] # 정답 인덱스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [34]:
# samples = m개
# classes = 3
# dim = 4

# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## Training with F.cross_entropy

In [35]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    # cost
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z,y_train)
    
    # H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## High-level Implementation with nn.Module

In [38]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3) # 4 -> 3
        
    def forward(self, x):
        return self.linear(x) # (m,4) -> (m,3)

In [39]:
model = SoftmaxClassifierModel()

In [40]:
# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost
    cost = F.cross_entropy(prediction, y_train)
    
    # H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 20 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch   20/1000 Cost: 0.904911
Epoch   40/1000 Cost: 0.794997
Epoch   60/1000 Cost: 0.746041
Epoch   80/1000 Cost: 0.714160
Epoch  100/1000 Cost: 0.689894
Epoch  120/1000 Cost: 0.669843
Epoch  140/1000 Cost: 0.652444
Epoch  160/1000 Cost: 0.636856
Epoch  180/1000 Cost: 0.622571
Epoch  200/1000 Cost: 0.609258
Epoch  220/1000 Cost: 0.596692
Epoch  240/1000 Cost: 0.584709
Epoch  260/1000 Cost: 0.573192
Epoch  280/1000 Cost: 0.562051
Epoch  300/1000 Cost: 0.551218
Epoch  320/1000 Cost: 0.540640
Epoch  340/1000 Cost: 0.530274
Epoch  360/1000 Cost: 0.520087
Epoch  380/1000 Cost: 0.510050
Epoch  400/1000 Cost: 0.500141
Epoch  420/1000 Cost: 0.490341
Epoch  440/1000 Cost: 0.480634
Epoch  460/1000 Cost: 0.471007
Epoch  480/1000 Cost: 0.461448
Epoch  500/1000 Cost: 0.451947
Epoch  520/1000 Cost: 0.442496
Epoch  540/1000 Cost: 0.433086
Epoch  560/1000 Cost: 0.423713
Epoch  580/1000 Cost: 0.414369
Epoch  600/1000 Cost: 0.405051
Epoch  620/1000 Cost: 0.395754
Epoch  6